# 0. Generating some time series
# 1. Box Cox and Derivation
# 2. Test Dickey-Fuller
# 3. Autocorrelation plot
# 4. Moving Average 
 - ## 4.1. Simple
- ## 4.2. Weighted Moving Average
- ## 4.3. Exponential Moving Average
- ## 4.4. Double Exponential Moving Average
- ## 4.5. Holt-Winters
# 5. How to define the params for Holt-Winters
# 6. Cross-validation in time series
# 7. models AR, ARMA, ARIMA etc
# 8. models ARCH, GARCH
# 9. Forecast using classic ML
# 10.
# 11.
# 12.
# 13.
# 14.
# 15.

# 0. Generating some time series
- $y_t = E$
- $y_t = y_{t-1} + \epsilon_{t}$
- $y_t = c + \sum_{i=1}^P a_i y_{t-i} + \epsilon_t$

In [ ]:
# 1. noise
randser = np.random.normal(size=1000)


#_______________
# 2. Random Walk
# Random Walk way 1
n_samples = 100
eps = np.random.normal(size=n_samples)
x = [0 + eps[0]]
for i in range(1, n_samples):
    x.append(x[i-1] + eps[i])



# Random Walk way 2
x = np.random.normal(size=1000)
x = np.cumsum(x) # cumsum - кумулятивная сумма [1, 2, 3, 4] -> [1, 3, 6, 10]



#_______________
# 3. AR simulation - регрессия второго порядка
np.random.seed(1)
n_samples = int(1000)
a1 = 0.6 # вес для лага 1 (t-1)
a2 = 0.3 # вес для лага 2 (t-2)
c = 0

x = w = np.random.normal(size=n_samples)

# регрессия второго порядка
for t in range(n_samples):
    x[t] = a1*x[t-1] + a2*x[t-2] + w[t] + c




#_______________
# 4. statsmodels provides the function 'arma_generate_sample' to generate random samples from an autoregressive moving average (ARMA) model. 
from statsmodels.tsa.arima_process import arma_generate_sample

max_lag = 30

n = int(5000) # lots of samples to help estimates ("много образцов для помощи в оценках")
burn = int(n/10) # number of samples to discard before fit

alphas = np.array([0.5, -0.25])
betas = np.array([0.5, -0.3])
ar = np.r_[1, -alphas] # объединенине массивов [1] и [0.5, -0.25]
ma = np.r_[1, betas]

arma22 = arma_generate_sample(ar=ar, ma=ma, nsample=n, burnin=burn)

# 1. Box Cox and Derivation (for getting a stationary time series)

In [ ]:
# 1. Box Cox Преобразование Бокса-Кокса часто применяется для нормализации данных или для стабилизации дисперсии
from scipy.stats import boxcox  
from scipy.special import inv_boxcox # для обратного преобразования

all_series = {
    "Monthly sales of company X": sales_of_company_x["Count"],
    "International airline passengers: monthly totals in thousands": airlines_passengers["Count"]
}

# boxcox(x, lmbda=None, alpha=None)  lmbda (необязательный)  - степень трансформации;  alpha: (необязательный) уровень значимости для доверительного интервала lmbda. 
series = boxcox(
    x=all_series["International airline passengers: monthly totals in thousands"], 
    lmbda=0,
    alpha=None
)


inv = inv_boxcox(series, lmbda=0)

In [ ]:
# 2. Derivation
import numpy as np 

# np.diff для вычисления разностей между элементами массива вдоль указанной оси. 
# для анализа временных рядов, вычисления производных или обнаружения изменений между последовательными значениями.
# np.diff(a, n=1, axis=-1) n: (по умолчанию 1) количество разностей, которые нужно вычислить. Например, n=2 вернет разности второго порядка.
# axis: (по умолчанию -1) ось, вдоль которой вычисляются разности.
series = np.diff(series, 1)


# or 
series = series[1:] - series[:-1]
# or
series = series[12:] - series[:-12]



In [ ]:
# example of reverting values into original way
init_list = [1,4,2,5,2,2]
init_list_bx = boxcox(init_list, 0)
init_list_diff = np.diff(init_list_bx, 1)

# Inverse operation for diff
initial_value = init_list_bx[0]
rev_dif = np.insert(np.cumsum(init_list_diff)+[initial_value]*len(init_list_diff), 0, initial_value)

# Inverse operation for Box-Cox
rev_bx = inv_boxcox(rev_dif, 0)

# 2. Test Dickey-Fuller (to test if the time series is stationary)

In [ ]:
from statsmodels.tsa.stattools import adfuller

adfuller(x, maxlag=None, regression='c', autolag='AIC', store=False, regresults=False)
# x: список, массив NumPy или серия pandas с временным рядом.
# maxlag: максимальное количество лагов
# regression: тип регрессии: тут 'c' для включения константы но есть и много других напр., 'ctt' для включения константы, линейного и квадратичного трендов,
# autolag: метод автоматического выбора лагов, тут 'AIC' для критерия Акаике
# store: если True, возвращает дополнительные результаты в виде класса с атрибутами.
# regresults: если True, сохраняет результаты промежуточной регрессии.


dftest = adfuller(timeseries, autolag='AIC')


# example of method
################
 # Dickey-Fuller
##################
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

# call method
test_stationarity(all_series["International airline passengers: monthly totals in thousands"])

In [ ]:
# **!!** HELPER METHOD for plotting
# helper method that calls test_stationarity() and plot Autocorrelation and Quantil-Quantil graph
def tsplot(y, lags=None, figsize=(14, 8), style='bmh'):
    plt.clf()
    test_stationarity(y)
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (5, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))
        qq_ax = plt.subplot2grid(layout, (4, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')

        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05) # автокорреляция
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05) # частичная автокорреляция
        sm.qqplot(y, line='s', ax=qq_ax) # график квантиль-квантиль
        
        plt.tight_layout()
        plt.show()
    return

In [ ]:
# **!!** HELPER METHOD for defining the seasonality SEASONALITY
def plot_ts_and_points(ts, start_point, step):
    plt.clf()
    if not isinstance(ts, pd.Series):
        ts = pd.Series(ts)
    new_series = [None for i in range(len(ts))]
    for i in range(len(ts)):
        pos = start_point + step * i
        if pos >= len(ts):
            break
        new_series[pos] = ts[pos]
    new_series = pd.Series(new_series)
    
    with plt.style.context('bmh'):
        plt.figure(figsize=(16, 8))
        ts_ax = plt.axes()
        ts.plot(ax=ts_ax, color='blue')
        new_series.plot(ax=ts_ax, style='ro')
        plt.show()

# 3. Autocorrelation plot

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt

# statsmodels.api предоставляет высокоуровневый доступ ко всем функциям библиотеки statsmodels. Обычно используется для общей статистики и моделирования.
# statsmodels.tsa.api предоставляет доступ к функциям и классам, специфичным для анализа временных рядов. 
# Это включает в себя модели ARIMA, тесты на стационарность, функции для построения автокорреляционных графиков и многое другое.

# plot_acf - Автокорреляционная функция - идентифицирует зависимость значений временного ряда на разных лагах.
smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)

# plot_pacf - Частичная автокорреляционная функция - показывает корреляцию значений временного ряда с лагами, устранение влияния промежуточных лагах.
smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)


#  метод для построения коррелорграммы 
def tsplot(y, lags=None, figsize=(14, 8), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (4, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')

        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)

        plt.tight_layout()
    return


tsplot(all_series["International airline passengers: monthly totals in thousands"])

# 4. Moving Average 

## 4.1.
$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k-1}_{n=0} y_{t-n}$

In [ ]:
# 1. method for calc the moving average
def moving_average(series, n):
    if not isinstance(series, pd.Series):
        series = pd.Series(series)
    return series.rolling(n).mean()

# call moving_average with the window equals 7
ser = moving_average(sales_of_company_x["Count"], 7)


# 2. predict value using the moving average approach 
def predict(series, N, n_pred):
    new_series = series.copy()
    len_series = len(series)
    for _ in range(n_pred):
        new_series[len_series+_] = int(pd.Series([new_series[-N:].mean()]))
    return new_series

# predict next 50 points 
series_pred = predict(sales_of_company_x["Count"], 7, 50)

## 4.2. Weighted Moving Average

$\hat{y}_{t} = \displaystyle\sum^{k}_{n=1} \omega_n y_{t+1-n}$

$\displaystyle\sum^{k}_{n=1} {\omega_n} = 1$

In [ ]:
# method for calc the weighted moving average
def weighted_moving_average(series, n, weights):
    if not isinstance(weights, np.ndarray):
        weights = np.array(weights)
    if not isinstance(series, pd.Series):
        series = pd.Series(series)
    wma = series.rolling(n).apply(lambda s: (s * weights).sum() / weights.sum(), raw=True) # тут / weights.sum() чтоб сделать веса (0,1]
    return wma


# call weighted_moving_average with the window equals 7 and weights = [1,1,2,3,5,8,13]
wma = weighted_moving_average(sales_of_company_x["Count"], 7, [1,1,2,3,5,8,13])

# 4.3. Exponential Moving Average
$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} $

In [ ]:
# 1. method for calc the exponential moving average
def exponential_moving_average(series, alpha):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return pd.Series(result)

# call exponential_moving_average with the alpha equals 0.2
ema = exponential_moving_average(sales_of_company_x["Count"], 0.2)


# 2. method exponential_moving_average + Predictions 
def exponential_moving_average(series, alpha, n_pred=None):
    result = [series[0]]
    for n in range(1, len(series)):
        result.append(alpha * series[n-1] + (1 - alpha) * result[n-1])
    if not n_pred:
        return pd.Series(result)

    len_series = len(series)
    predictions = [alpha * series[len_series - 1] + (1 - alpha) * result[-1]]

    for _ in range(n_pred):
        res = alpha * predictions[-1] + (1 - alpha) * result[-1]
        result.append(res)
        predictions.append(res)
        series[len_series+_] = res
    
    return series, pd.Series(result)

# 4.4. Double Exponential Moving Average
$\hat{y}_t=l_t + s_t$

$\hat{y}_t=\alpha y_t + (1-\alpha)\hat{y}_{t-1}$

$\hat{l}_t = \alpha y_t + (1-\alpha)(l_{t-1} + s_{t-1})$

$l_t = \alpha y_t + (1-\alpha)(l_{t-1} + s_{t-1})$

$s_t = \beta (l_t - l_{t-1}) + (1 - \beta) s_{t-1}$

<b> настройка параметров $\alpha$ и $\beta$ и  может порой давать самые причудливые результаты. $\alpha$ отвечает за сглаживание ряда вокруг тренда, $\beta$ - за сглаживание самого тренда. Чем больше значения, тем более значимыми будут последние наблюдения и менее значимой будет история.

In [ ]:
# 1. method for calc the double exponential moving average
def double_ema(series, alpha, beta):
    result = [series[0]]
    level, trend = series[0], series[1] - series[0]
    for n in range(1, len(series)):
        value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return pd.Series(result)


# for plot graph 
def plot_dema(alpha, beta):
    dema = double_ema(sales_of_company_x["Count"], alpha, beta)
    with plt.style.context('bmh'):
        plt.figure(figsize=(14, 8))
        plt.plot(sales_of_company_x["Count"], color='blue',label='original')
        plt.plot(dema, color='red', linewidth='4', label='DEMA')
        plt.title("alpha={}, beta={}".format(alpha, beta))
        plt.legend()

# calling method plot_dema
plot_dema(0.2, 0.2)


# 2. method exponential_moving_average + Predictions 
def double_ema_with_preds(series, alpha, beta, n_preds):
    result = [series[0]]
    level, trend = series[0], series[1] - series[0]
    for n in range(1, len(series)):
        value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)

    len_series = len(series)
    preds = []
    for n in range(n_preds):
        value = result[-1]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
        preds.append(level+trend)
        series[len_series+n] = level+trend

    return series, pd.Series(result)

# 4.5. Holt-Winters
### Важно - метод применим только в случае сезонности. Если ее нет - метод хольта-винтерса не даст хороших результатов;

$l_t = \alpha(y_t - p_{t-\tau}) + (1-\alpha)(l_{t-1} + s_{t-1})$


$s_t = \beta(l_t - l_{t-1}) + (1-\beta)s_{t-1}$

$p_t = \gamma(y_t - l_t) + (1-\gamma)p_{t-\tau}$

$\hat{y}_{t+m} = l_x + s_t + p_{t-\tau+1+(m-1)mod\tau}$

In [ ]:
def initial_trend(series, season_len):
    """
    This function calculates the initial trend of the time series. 
    The trend is the average of the differences between each value and the value season_len steps before it, normalized by season_len.
    """
    return sum([float(series[i + season_len] - series[i]) / season_len]) / season_len


def initial_seasonal_components(series, slen):
    """
    It iterates over each season, calculates the average, and then determines how much each point deviates from these averages.
    
    inputs:
        seasonals stores the seasonal components.
        season_averages stores the average of each season.
    """
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals


def triple_exponential_smoothing(series, slen, alpha, beta, gamma, n_preds):
    """
    The function initializes the smoothed value, trend, and seasonal components, 
    then iterates over the time series to update these values using the smoothing equations. 
    It forecasts future values by extending the series and applying the trend and seasonal components.

    inputs:
        series is the time series data.
        slen is the length of a season (e.g., 12 for monthly data with yearly seasonality).
        alpha, beta, and gamma are the smoothing parameters.
        n_preds is the number of future points to predict.
    """
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result


def plot_tema(alpha, beta, gamma, ser=sales_of_company_x["Count"], ser_to_plot=sales_of_company_x["Count"], n_preds=24):
    tema = triple_exponential_smoothing(ser, 12, alpha, beta, gamma, n_preds)
    with plt.style.context('bmh'):
        plt.figure(figsize=(14, 8))
        plt.plot(ser_to_plot, color='blue',label='original')
        plt.plot(tema, color='red', linewidth='4', label='TEMA')
        plt.title("alpha={}, beta={}, gamma={}".format(alpha, beta, gamma))
        plt.legend()


plot_tema(0.1, 0.1, 0.2)

# 5. How to define the params for Holt-Winters
$RMSE =  \sqrt {1/n \sum^N_{i=1} (\hat{y}_i - y_i)^2}$

In [2]:
from scipy.optimize import minimize 
from sklearn.metrics import mean_squared_error


# Init data
series = sales_of_company_x["Count"]
train, test, val = series[:60], series[60:70], series[70:]


# Create scoring method 
def mse(X):
    alpha, beta, gamma = X
    result = triple_exponential_smoothing(train, 12, alpha, beta, gamma, len(test)) # triple_exponential_smoothing (see case 4.5.)
    predictions = result[-len(test):]
    error = mean_squared_error(predictions, test)
    return error

# Call the minimize from scipy for looking for optimize alpha, beta and gamma
opt = minimize(mse, x0=[0,0,0], method="L-BFGS-B", bounds = ((0, 1), (0, 1), (0, 1)))

# Get opt perams 
alpha_opt, beta_opt, gamma_opt = opt.x
print(opt)

# Use gotten params in plot_tema (see case 4.5.)
plot_tema(alpha_opt, beta_opt, gamma_opt, ser=train, ser_to_plot=series[:70], n_preds=len(test))

# 6. Cross-validation in time series

Проблема кросс-валидации на временных рядах состоит в том, что случайно перемешивать в фолдах значения всего ряда нельзя. Т.к. он имеет временную структуру, и ее надо сохранять (иначе потеряются все взаимосвязи наблюдений);

Будем делать кросс-валидацию на скользящем окне.

Суть достаточно проста:

Берем t измерений
Делаем прогноз на n измерений вперед и считаем ошибку
Берем t+n измерений
Делаем прогноз на n измерений вперед и считаем ошибку
Берем t+2*n измерений
Делаем прогноз на n измерений вперед и считаем ошибку ...

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from scipy.optimize import minimize 
from sklearn.metrics import mean_squared_error

# Init data
train, val = series[:65], series[65:]

# Create scoring method with cross-validation
def mse_cross_val(X):
    alpha, beta, gamma = X
    split = TimeSeriesSplit(n_splits=3) 
    errors = []
    
    for train_split, test_split in split.split(train):
        train_split_ts = train.iloc[train_split]
        test_split_ts = train.iloc[test_split]
#         print(train_split_ts)
#         print(test_split_ts)
        result = triple_exponential_smoothing(train_split_ts, 12, alpha, beta, gamma, len(test_split)) # triple_exponential_smoothing (see case 4.5.)
        predictions = result[-len(test_split_ts):]
        error = mean_squared_error(predictions, test_split_ts)
        errors.append(error)
    print(f'{np.mean(np.array(errors))}------------')
    return np.mean(np.array(errors))


# Call the minimize from scipy for looking for optimize alpha, beta and gamma
opt = minimize(mse_cross_val, x0=[0,0,0], method="Nelder-Mead", bounds = ((0, 1), (0, 1), (0, 1)))

# Get opt perams 
alpha_opt, beta_opt, gamma_opt = opt.x
print(opt)

# Use gotten params in plot_tema (see case 4.5.)
plot_tema(alpha_opt, beta_opt, gamma_opt, ser=train, ser_to_plot=series, n_preds=len(val))

# 7. AR, ARMA, ARIMA etc

1) <b>AR</b> (Авторегрессионная модель) — модель временных рядов, в которой значения временного ряда в данный момент линейно зависят от предыдущих значений этого же ряда. Авторегрессионный процесс порядка p (AR(p)-процесс) определяется следующим образом

$y_t = c + \sum_{i=1}^P a_i y_{t-i} + \epsilon_t$

где 
- $c$ - константа 
- $\sum_{i=1}^P a_i y_{t-i}$ - сумма взешенных предыдущих значений
- $P$ - лаг или порядок авторегрессии AR (сколько шагов назад)
- $\epsilon_t$ - белый шум

2) <b>ARMA(p, q)</b> - модель ARMA(p, q) представляет собой соединение двух моделей:
- AR(P) - авторегрессии на знанениях временного ряда
- MA(q) - скользящее среднее на ошибках первой модели

<i>p.s. В statsmodel в последней версии убрали класс ARMA оставив только ARIMA, поэтому тут описание для ARMA но код на ARIMA. </i>

<i>p.s.s. По сути модель ARMA можно получить с помощью ARIMA , занулив второй параметр ARIMA(p,d=0,q)</i>

AR(p) пытается предсказать "значение" временного ряда, а MA(q) пытается поймать шоковые явления, наблюдаемые в оставшемся случайном шуме.

$y_t = \sum_{i=1}^P a_i x_{t-i} + \sum_{i=1}^Q b_i \epsilon_{t-i} + \epsilon_t + c$

- $\sum_{i=1}^P a_i x_{t-i}$ - обычная авторегрессия
- $ \sum_{i=1}^Q b_i \epsilon_{t-i}$ - скользящее среднее на ошибках AR

3) <b>ARIMA</b> - естественное расширение модели ARMA. Как мы уже хорошо знаем - многие временные ряды не стационарны, но они могут такими стать в результате операции дифференцирования. В модели ARIMA "дифференцирование" (в количестве d-раз) вносится в саму модель


$\delta^p y_t = c + \sum_{i=1}^p a_i \delta^d y_{t-i} + \sum_{j=1}^q b_j \epsilon_{t-j} + \epsilon_t$


4) <b>SARIMA</b> - арима с учетом сезонности
5) <b>ARIMAX/SARIMAX</b> - ARIMAX использует внешние переменные для объяснения временных изменений в ряду данных, а SARIMAX дополнительно учитывает сезонные колебания, что полезно для данных с регулярными сезонными паттернами и внешними воздействиями.

In [ ]:
import statsmodels.tsa.ar_model as arm     # arm.AutoReg ; arm.ar_select_order
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima_model import ARMA

In [ ]:
# 1) AR
import statsmodels.tsa.ar_model as arm

# The 'arm.AutoReg' class is used to fit an autoregressive model to a time series. 
mdl = arm.AutoReg(x, lags=3, trend='n').fit()

# 'arm.ar_select_order' is used to determine the optimal number of lags based on criteria (like AIC, BIC, etc.).
est_order = arm.ar_select_order(x, maxlag=3, trend='n', ic='aic')

print('alpha estimate: {}'.format(mdl.params[:len(est_order.ar_lags)]))
print('best lag order = {}'.format(len(est_order.ar_lags)))

# the optimal number of lags
est_order.ar_lags

# fitted coefficients
mdl.params


# resid - вернет значения остатков (из реального time series вычитается предсказанный)
# по resid можно построить график остатков (в идеале который должен быть просто линией в нуле)
# задача состоит в том, чтоб график остатков был !!стационарен!!, таким образом мы будем знать что мы
# вытащили из временного ряда все что можно было)
tsplot(mdl.resid, lags=30) # tsplot find upper

In [ ]:
# 2) ARMA (p,q) ARIMA(p, d, q)
from statsmodels.tsa.arima_model import ARMA # не поддерживается в пользу statsmodels.tsa.arima.model.ARIMA
from statsmodels.tsa.arima.model import ARIMA # I  - значит также проводится дифференцирование 


# looking for the best params for ARMA or ARIMA using AIC - Информационный критерий Акаике
# p, d, q - гипер параметры => их можно подбирать. Как выбрать лучшие? AIC - Информационный критерий Акаике
best_aic = np.inf 
best_order = None
best_mdl = None

for i in range(5):
    for d in range(5):  # for ARMA this param has to be 0
        for j in range(5):
            try:
                tmp_mdl = ARIMA(series, order=(i, d, j), trend='c').fit(method='innovations_mle')
                tmp_aic = tmp_mdl.aic
                if tmp_aic < best_aic:
                    best_aic = tmp_aic
                    best_order = (i, d, j)
                    best_mdl = tmp_mdl
            except: continue


print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))


tsplot(best_mdl.resid, lags=30)

In [1]:
# График оригинального (стационарного) ряда , предсказания а также 95% доверительного интервала (интервал где с 95% долей вероятности окажется след. значение)
forecast = mdl.get_forecast(steps=20)
forecast_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

# Creating an index for the forecasted values
forecast_index = np.arange(len(series), len(series) + 20)

with plt.style.context(style='bmh'):
    plt.figure(figsize=(14,8))
    ax = plt.axes()
    plt.plot(series, color='red', label='original, stationary') # original
    plt.plot(mdl.predict(start=0, end=len(series)+20), color='blue', label='prediction, stationary') # mdl.predict

    # Plotting the only forecasted values
    #plt.plot(forecast_index, forecast_values, color='blue', label='prediction, stationary')

    # Plotting the confidence intervals
    plt.fill_between(forecast_index, 
                     confidence_intervals[:, 0], 
                     confidence_intervals[:, 1], 
                     color='blue', alpha=0.2, label='95% confidence intervals')
    plt.legend()
    

# 8. ARCH and GARCH

## 1. ARCH - autoregressive conditional heteroskedasticity
- <b>ARCH</b> - модель авторегрессии (AR) условной герероскеданичности (CH)

<b>AutoRegressive Conditional Heteroscedasticit</b> - пытаемся объяснить дисперсию в ряде через предыдущие значения (применяя к ним AR).
- т.е. модель смотрит не просто значения, но также учитывает дисперсию этих значений
- этой моделью ставится первичная цель предсказать дисперсию


<p></p>

- <b>Гетероскедастичность</b> - когда дисперсия временного ряда непостоянна.
- <b>Гомоскедастичность</b> - когда дисперсия временного ряда постоянна для всех значений независимых переменных.

Пусть временной ряд представляется в таком виде:

$u_t = \epsilon_t * \sqrt{\alpha_0 + \sum_{i=1}^q \alpha_i u_{t-i}^2}$

Тогда условная дисперсия ряда будет равна

$\sigma_t^2 = V(u_t | u_{t-1}, ..., u_{t-q}) = \alpha_0 + \sum_{i=1}^q \alpha_i u_{t-i}^2$

Получили модель ARCH(q) условной дисперсии. Требуем, чтобы все коэффициенты были больше 0 (иначе может получится отрицательная дисперсия)

## 2. GARCH хорошо учитыавет дисперсию в моделях, где дисперсия ведет себя как временной ряд

Добавляем зависимость от прошлых значений самой условной дисперсии. Получаем модель GARCH(p, q)

$\sigma_t^2 = V(u_t | u_{t-1}, ..., u_{t-q}) = \alpha_0 + \sum_{i=1}^q \alpha_i u_{t-i}^2 + \sum_{j=1}^p \beta_j \sigma_{t-j}^2$

GARCH - по сути - модель ARMA примененная к дисперсии ряда


p.s. <b> Важно отметить, что изученные модели (AR, ARMA, ARIMA, ARCH, GARCH) довольно плохи в предсказании, но с ними можно строить модель или принимать решение на основе доверительных интервалов.</b>

In [ ]:
from arch import arch_model

In [ ]:
# the method of looking for the best params

def _get_best_model(TS):
    best_aic = np.inf 
    best_order = None
    best_mdl = None

    for i in range(5):
        for d in range(5):
            for j in range(5):
                try:
                    tmp_mdl = arch_model(series, p=i, o=d, q=j, dist='StudentsT').fit(update_freq=5, disp='off') 
                    #tmp_mdl = ARIMA(TS, order=(i, d, j)).fit(method='innovations_mle') # параметры полученные с помощью ARIMA можно применить и для GARCH
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (i, d, j)
                        best_mdl = tmp_mdl
                except: continue
    print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))                    
    return best_aic, best_order, best_mdl


aic, order, mdl = _get_best_model(series)

# график остатков
tsplot(res.resid)

In [ ]:
order = [4,2,4]

p_ = order[0]
o_ = order[1]
q_ = order[2]

# Using student T distribution usually provides better fit
am = arch_model(new_series, p=p_, o=o_, q=q_, dist='StudentsT')
res = am.fit(update_freq=5)
print(res.summary())

# 9. Forecast using classic ML

### "По простому"
##### С предсказанием же лучше справится та же Линейная регрессия от sklearn, т.к. в ней реализовано аналитическое решение и она гарантированно вернет лучшее решение.

In [ ]:
from sklearn.linear_model import LinearRegression


def prepareData(data, lag_start=5, lag_end=20, test_size=0.15):
    
    data = pd.DataFrame(data.copy())
    
    # считаем индекс в датафрейме, после которого начинается тестовый отрезок
    test_index = int(len(data)*(1-test_size))
    
    # добавляем лаги исходного ряда в качестве признаков
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.Count.shift(i)
        """
                    e.g.
                         Month      Count  lag_1   lag_2    lag_3
                    0    1949-01    112    NaN     NaN      NaN
                    1    1949-02    118    112.0   NaN      NaN
                    2    1949-03    132    118.0   112.0    NaN
        """
        
    data = data.dropna()
    data = data.reset_index(drop=True)
    data = data.drop(["Month"], axis=1)
     
    # разбиваем весь датасет на тренировочную и тестовую выборку
    X_train = data.loc[:test_index].drop(["Count"], axis=1)
    y_train = data.loc[:test_index]["Count"]
    X_test = data.loc[test_index:].drop(["Count"], axis=1)
    y_test = data.loc[test_index:]["Count"]
    
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = prepareData(series, lag_start=1, lag_end=20, test_size=0.3)


lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X_test)

plt.figure(figsize=(14, 8))
plt.plot(prediction, "r", label="prediction")
plt.plot(y_test.values, label="actual")
plt.legend(loc="best")
plt.title("Linear regression")
plt.grid(True);